In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [2]:

# paths for train and test directories
train_dir = r'F:\U n i v e r s i t y\5TH - S E M E S T E R\AI\Artificial - Intellligennce\student-engagemnet-tracking-application\student-engagement-tracking\backend\engangement dataset\train'
test_dir = r'F:\U n i v e r s i t y\5TH - S E M E S T E R\AI\Artificial - Intellligennce\student-engagemnet-tracking-application\student-engagement-tracking\backend\engangement dataset\test'

# Engagement labels
engaged_labels = ['happy', 'neutral', 'surprise', 'lookingAtCamera', 'not-yawn']
not_engaged_labels = ['angry', 'disgust', 'fear', 'sad', 'lookingAwayFromCamera', 'yawn']
all_labels = engaged_labels + not_engaged_labels


In [ ]:
def load_and_preprocess_images(data_dir, target_size=(224, 224)):
    images, labels = [], []

    for label_name in all_labels:
        folder_path = os.path.join(data_dir, label_name)
        label = 1 if label_name in engaged_labels else 0  # 1: engaged, 0: not engaged

        if os.path.exists(folder_path):
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                image = cv2.imread(img_path, cv2.IMREAD_COLOR)

                if image is not None:
                    image = cv2.resize(image, target_size)
                    images.append(image)
                    labels.append(label)
        else:
            print(f"Warning: Directory {folder_path} not found.")

    images = np.array(images, dtype='float32') / 255.0
    labels = np.array(labels, dtype='int')

    return images, labels

# Load and preprocess dataset
X_train, y_train = load_and_preprocess_images(train_dir)
X_test, y_test = load_and_preprocess_images(test_dir)

print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Labels shape: {y_test.shape}")


Handle class imbalnce

In [ ]:
from sklearn.utils import class_weight

# Compute class weights 
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class Weights:", class_weights_dict)


In [12]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print(f"One-hot encoded labels shape: {y_train.shape}, {y_test.shape}")


225/225 - 5s - 21ms/step - accuracy: 0.7671 - loss: 0.8150
Test accuracy: 76.71%


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

def create_efficientnet_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and summarize model
model = create_efficientnet_model()
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, reduce_lr]
)

# Save the model
model.save('student-engagement_model.h5')


In [ ]:
# Evaluate on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import numpy as np

# Predict and calculate confusion matrix
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, y_pred)

# Display Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Engaged", "Engaged"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=["Not Engaged", "Engaged"]))


In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)
